In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import json

In [5]:
df = pd.read_csv("preprocessed_dataset/preprocessed_train.csv")

In [6]:
df['SEX'] = df['SEX'].map({'M': 1, 'F': 0})

In [7]:
min_max_scaler = MinMaxScaler()
df['AGE'] = min_max_scaler.fit_transform(df[['AGE']])

In [9]:
def obtain_evidences():
    with open("huggingface_dataset/ddxplus/release_evidences.json", "r") as f:
        release_evidences = json.load(f)
    set_evidences = []
    for key, value in release_evidences.items():
        if len(value['possible-values']) > 0:
            for val in value['possible-values']:
                set_evidences.append(f'{key}_@_{val}')
        else:
            set_evidences.append(f'{key}')
    return set_evidences

In [10]:
def obtain_conditions():
    with open("huggingface_dataset/ddxplus/release_conditions.json") as f:
        release_conditions = json.load(f)
    return [a for a in release_conditions.keys()]

In [11]:
features = ['AGE', 'SEX'] + obtain_evidences()
labels = obtain_conditions()

In [12]:
X_train = df[features].values

In [13]:
Y_train = df[labels].values

In [14]:
print(X_train.shape)
print(Y_train.shape)

(1025602, 974)
(1025602, 49)


In [15]:
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers

In [89]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU găsit: {gpu.name}")

GPU găsit: /physical_device:GPU:0


In [16]:
def elementwise_accuracy(y_true, y_pred, threshold=0.03):
    diffs = tf.abs(y_true - y_pred)
    all_close = tf.reduce_all(diffs <= threshold, axis=1)
    return tf.reduce_mean(tf.cast(all_close, tf.float32))

In [17]:
import tensorflow as tf

def top3_tolerant_accuracy(y_true, y_pred):
    top3_indices = tf.argsort(y_pred, direction='DESCENDING')[:, :3]
    batch_size = tf.shape(y_pred)[0]

    batch_indices = tf.range(batch_size)
    batch_indices = tf.reshape(batch_indices, (-1, 1))
    batch_indices = tf.tile(batch_indices, [1, 3])
    indices = tf.stack([batch_indices, top3_indices], axis=2)

    top3_true = tf.gather_nd(y_true, indices)
    top3_pred = tf.gather_nd(y_pred, indices)

    abs_diff = tf.abs(top3_true - top3_pred)
    correct = tf.reduce_all(abs_diff <= 0.05001, axis=1)
    return tf.reduce_mean(tf.cast(correct, tf.float32))

In [18]:
y_true = tf.constant([
    [0.0, 0.2, 0.6, 0.1, 0.1],
    [0.7, 0.1, 0.1, 0.05, 0.05]
], dtype=tf.float32)

y_pred = tf.constant([
    [0.05, 0.22, 0.5, 0.08, 0.07],
    [0.75, 0.08, 0.08, 0.04, 0.05]
], dtype=tf.float32)

acc = top3_tolerant_accuracy(y_true, y_pred)
print(acc)


tf.Tensor(0.5, shape=(), dtype=float32)


2025-04-07 17:57:36.750198: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-04-07 17:57:36.750258: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-04-07 17:57:36.750267: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2025-04-07 17:57:36.750302: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-07 17:57:36.750324: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
input_dim = len(features)
output_dim = len(labels)

model = models.Sequential([
    layers.Input(shape=(input_dim,), name='Input'),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(output_dim, activation='softmax', name='Output')
])

In [24]:
def combined_loss(y_true, y_pred):
    kl = tf.keras.losses.KLDivergence()(y_true, y_pred)
    top3 = 1.0 - top3_tolerant_accuracy(y_true, y_pred)
    return 0.999 * top3 + 0.0001 * kl

In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.KLDivergence(),
    metrics=[
        elementwise_accuracy,
        top3_tolerant_accuracy
    ]
)

In [26]:
model.fit(
    X_train, Y_train,
    epochs=10,
    batch_size=512
)

Epoch 1/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 44s 21ms/step - elementwise_accuracy: 3.9302e-05 - loss: 1.6035 - top3_tolerant_accuracy: 0.0708
Epoch 2/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 21ms/step - elementwise_accuracy: 2.8909e-04 - loss: 0.9331 - top3_tolerant_accuracy: 0.1030
Epoch 3/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 21ms/step - elementwise_accuracy: 7.2923e-04 - loss: 0.8566 - top3_tolerant_accuracy: 0.1149
Epoch 4/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - elementwise_accuracy: 0.0011 - loss: 0.8298 - top3_tolerant_accuracy: 0.1218
Epoch 5/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 21ms/step - elementwise_accuracy: 0.0012 - loss: 0.8217 - top3_tolerant_accuracy: 0.1241
Epoch 6/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 21ms/step - elementwise_accuracy: 0.0016 - loss: 0.8187 - top3_tolerant_accuracy: 0.1255
Epoch 7/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 43s 21ms/step - elementwise_accuracy: 0.0016 - loss: 0.8175 - top3_tolerant_accuracy: 0.1252
Epoch 8/10
2004/2004 ━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Crearea modelului
model = xgb.XGBClassifier(objective='multi:softmax', num_class=49, random_state=42)

# Antrenarea modelului
model.fit(X_train, Y_train)

# Predicția și evaluarea
y_pred = model.predict(X_train)
accuracy = accuracy_score(Y_train, y_pred)
print(f'Accuracy: {accuracy}')


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [      0       1       2 ... 2709076 2709077 2709078], got [0.00000000e+00 8.81165992e-05 9.51376338e-05 ... 9.98144722e-01
 9.98160638e-01 1.00000000e+00]

In [48]:
import torch
print(torch.__version__)
print(f"MPS disponibil: {torch.backends.mps.is_available()}")

2.6.0
MPS disponibil: True


In [49]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [51]:
class MLPModel(nn.Module):
    def __init__(self, input_dim, output_dim, l2_lambda=0.01):
        super(MLPModel, self).__init__()
        self.dense1 = nn.Linear(input_dim, 512)
        self.dropout1 = nn.Dropout(0.3)
        self.dense2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.dense3 = nn.Linear(256, 128)
        self.output = nn.Linear(128, output_dim)
        self.l2_lambda = l2_lambda

    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = self.dropout1(x)
        x = F.relu(self.dense2(x))
        x = self.dropout2(x)
        x = F.relu(self.dense3(x))
        x = self.output(x)
        return x

    def l2_regularization_loss(self):
        l2_loss = 0
        for param in self.parameters():
            l2_loss += torch.norm(param, p=2)
        return self.l2_lambda * l2_loss

In [56]:
model_mlp = MLPModel(input_dim, output_dim)
model_mlp.to(device=device)

MLPModel(
  (dense1): Linear(in_features=974, out_features=512, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dense2): Linear(in_features=512, out_features=256, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dense3): Linear(in_features=256, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=49, bias=True)
)

In [63]:
criterion = nn.KLDivLoss(reduction='batchmean')

In [64]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.01)

In [82]:
from torch.utils.data import DataLoader, TensorDataset
x_train_tensor = torch.tensor(X_train[:500000], dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train[:500000], dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [84]:
model_mlp.train()
for data, target in train_loader:
    data, target = data.to(device), target.to(device)
    
    optimizer.zero_grad()
    
    output = model_mlp(data)
    output_probabilities = F.softmax(output, dim=1)
    output_log_probabilities = F.log_softmax(output, dim=1)
    loss = criterion(output_log_probabilities, target)
    l2_loss = model_mlp.l2_regularization_loss()
    total_loss = loss + l2_loss
    total_loss.backward()
    optimizer.step()
    print(f'Loss: {loss.item()}, L2 Loss: {l2_loss.item()}')

Loss: 1.9547476768493652, L2 Loss: 0.3447197675704956
Loss: 1.977004051208496, L2 Loss: 0.3447197675704956
Loss: 2.1346991062164307, L2 Loss: 0.3447197675704956
Loss: 2.0044898986816406, L2 Loss: 0.3447197675704956
Loss: 1.7588226795196533, L2 Loss: 0.3447197675704956
Loss: 2.1007931232452393, L2 Loss: 0.3447197675704956
Loss: 2.0852982997894287, L2 Loss: 0.3447197675704956
Loss: 1.9676635265350342, L2 Loss: 0.3447197675704956
Loss: 2.1425154209136963, L2 Loss: 0.3447197675704956
Loss: 1.9719147682189941, L2 Loss: 0.3447197675704956
Loss: 1.9442415237426758, L2 Loss: 0.3447197675704956
Loss: 2.1176562309265137, L2 Loss: 0.3447197675704956
Loss: 2.3768954277038574, L2 Loss: 0.3447197675704956
Loss: 1.9029721021652222, L2 Loss: 0.3447197675704956
Loss: 2.05721378326416, L2 Loss: 0.3447197675704956
Loss: 2.100092649459839, L2 Loss: 0.3447197675704956
Loss: 1.9221172332763672, L2 Loss: 0.3447197675704956
Loss: 1.9993609189987183, L2 Loss: 0.3447197675704956
Loss: 1.9658374786376953, L2 Los